# Vertex AI - Feature Store

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `06 - Vertex AI - Training Pipeline and Serving`
    - for online prediction this used the endpoint defined in 06
    
**Resources**
- Based on:
    - https://cloud.google.com/vertex-ai/docs/featurestore/managing-featurestores
- API Documentation:
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/services.html
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/featurestore_service.html

**Overview**

<img src="architectures/statmike-mlops-07.png">


---
## Setup

Update Python library for aiplatform (Vertex AI)
- Restart the kernel: Menus > Kernel > Restart Kernel

In [1]:
!pip install --upgrade git+https://github.com/googleapis/python-aiplatform.git@main-test -q

Import Libraries

In [1]:
from google.cloud.aiplatform_v1beta1 import (FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, types)
from google.protobuf.duration_pb2 import Duration
from google.protobuf.timestamp_pb2 import Timestamp
from google.protobuf.field_mask_pb2 import FieldMask

Setup GCP Parameters

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

Clients for Feature Store:

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

In [4]:
clients['fs'] = FeaturestoreServiceClient(client_options=client_options)

In [5]:
clients['fs_olserve'] = FeaturestoreOnlineServingServiceClient(client_options=client_options)

In [6]:
BASE_RESOURCE_PATH = clients['fs'].common_location_path(PROJECT_ID, REGION)

---
## Feature Store Data model
Feature Store organizes data with the following 3 important hierarchical concepts:

Featurestore -> EntityType -> Feature

- **Featurestore**: the place to store your features
    - **EntityType**: under a Featurestore, an EntityType describes an object to be modeled, real one or virtual one.
        - **Feature**: under an EntityType, a feature describes an attribute of the EntityType

For the digits data used in these examples, the feature store will be called digits_featurestore.  The store has 1 entity type: images.  The features will be the pixels and the target values.

---
## Create Feature Store

In [7]:
FEATURESTORE_ID = 'digits_featurestore'

In [8]:
featurestore_lro = clients['fs'].create_featurestore(
    types.featurestore_service.CreateFeaturestoreRequest(
        parent = BASE_RESOURCE_PATH,
        featurestore_id = FEATURESTORE_ID,
        featurestore=types.featurestore.Featurestore(
            display_name="Featurestore for handwritten digits",
            online_serving_config=types.featurestore.Featurestore.OnlineServingConfig(
                fixed_node_count=3
            ),
        ),
    )
)

In [9]:
featurestore_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"

Use `get_featurestore` to see details of specified feature store:

In [ ]:
clients['fs'].get_featurestore(name=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID))

Use `list_featurestores` to see details of all feature stores:

In [8]:
clients['fs'].list_featurestores(parent=PARENT)

ListFeaturestoresPager<featurestores {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"
  create_time {
    seconds: 1630413385
    nanos: 264800000
  }
  update_time {
    seconds: 1630413385
    nanos: 628083000
  }
  etag: "AMEw9yMm3f2B1v40uRDYtsPuOIPlxK7jEsV23jOdf5aLuWGb2vnHUciaKVa4rgixh8Rr"
  online_serving_config {
    fixed_node_count: 3
  }
  state: STABLE
}
>

---
## Create Entity Type

In [17]:
ENTITYTYPE_ID = 'person'

In [13]:
entitytype_lro = clients['fs'].create_entity_type(
    types.featurestore_service.CreateEntityTypeRequest(
        parent=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id = ENTITYTYPE_ID,
        entity_type=types.entity_type.EntityType(
            description="person entity",
            monitoring_config=types.featurestore_monitoring.FeaturestoreMonitoringConfig(
                snapshot_analysis=types.featurestore_monitoring.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=900),  # 15 minutes
                ),
            ),
        ),
    )
)

In [14]:
entitytype_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person"
etag: "AMEw9yOE1pqoZ9k-ToNi-r4HstwbqM5sNuPycGpGEwCgvB0nBi6c"

Use `list_entity_types` to see details of all entity types:

In [10]:
clients['fs'].list_entity_types(parent=PARENT+'/featurestores/{}'.format(FEATURESTORE_ID))

ListEntityTypesPager<entity_types {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person"
  description: "person entity"
  create_time {
    seconds: 1630413396
    nanos: 748489000
  }
  update_time {
    seconds: 1630590074
    nanos: 797046000
  }
  etag: "AMEw9yPHNRQgn_h0IfizOZCe_xSNDUsdR-TAZQ1zdmTfKD86N7hHMl3uthOL8YSR-rBI"
  monitoring_config {
    snapshot_analysis {
      monitoring_interval {
        seconds: 86400
      }
      monitoring_interval_days: 1
    }
  }
}
>

---
## Create Features

Get the schema of the data source for new features:

In [16]:
%%bigquery schema
SELECT * 
FROM `statmike-mlops.digits.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE table_name = 'digits_source'

Downloading: 100%|██████████| 66/66 [00:01<00:00, 54.34rows/s]


In [17]:
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_source,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_source,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_source,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_source,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_source,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
61,statmike-mlops,digits,digits_source,p61,p61,FLOAT64,None
62,statmike-mlops,digits,digits_source,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_source,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_source,target,target,INT64,None


Prepare a request for `batch_create_features`:
- specification for the features, data type and descriptions ....

In [18]:
REQUESTS = []
for i in range(schema.shape[0]):
    
    if schema['data_type'][i] == 'STRING': value_type = types.feature.Feature.ValueType.STRING
    elif schema['data_type'][i] == 'INT64': value_type = types.feature.Feature.ValueType.INT64
    elif schema['data_type'][i] == 'FLOAT64': value_type = types.feature.Feature.ValueType.DOUBLE
    
    if schema['description'][i] == None: description = schema['column_name'][i]
    else: description = schema['description'][i]
    
    REQUESTS.append(
        types.featurestore_service.CreateFeatureRequest(
            feature=types.feature.Feature(
                value_type = value_type,
                description = description,
                # optional, monitoring_config here as override, otherwise it inherits from entity_type
            ),
            feature_id = schema['column_name'][i].lower(),
        )    
    )

In [19]:
batchfeatures = clients['fs'].batch_create_features(
    parent = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    requests = REQUESTS,
)

In [20]:
list(item.name for item in batchfeatures.result().features)

['projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p0',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p1',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p2',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p3',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p4',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p5',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p6',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p7',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/pers

---
## Search Features
Search goes across all Feature Stores and Entity Types.

Also, use the list_features function to list all.

In [11]:
# return the first feature:
list(clients['fs'].search_features(location=BASE_RESOURCE_PATH))[0]

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p0"
description: "p0"
create_time {
  seconds: 1630413411
  nanos: 932022000
}
update_time {
  seconds: 1630413411
  nanos: 932022000
}

In [12]:
# find all features with INT64 value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="value_type=INT64")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/target"
 description: "target"
 create_time {
   seconds: 1630413412
   nanos: 57643000
 }
 update_time {
   seconds: 1630413412
   nanos: 57643000
 }]

In [13]:
# find all features of the form p6* with DOUBLE value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="feature_id:p6* AND value_type=DOUBLE")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p6"
 description: "p6"
 create_time {
   seconds: 1630413411
   nanos: 942106000
 }
 update_time {
   seconds: 1630413411
   nanos: 942106000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p60"
 description: "p60"
 create_time {
   seconds: 1630413412
   nanos: 49458000
 }
 update_time {
   seconds: 1630413412
   nanos: 49458000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p61"
 description: "p61"
 create_time {
   seconds: 1630413412
   nanos: 50986000
 }
 update_time {
   seconds: 1630413412
   nanos: 50986000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/person/features/p62"
 description: "p62"
 create_time {
   seconds: 1630413412
   nanos: 53342000
 }
 update_time {
   sec

---
## Import Feature Values
- BigQuery (THIS DEMO)
- Avro
- CSV

Prepare a source table with timestamp (update_time) and unique id's (image_id) for each entity

In [24]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops.digits.digits_featurestore_import` AS
WITH A AS (SELECT GENERATE_UUID() unique_id, target_OE as target_oe, CAST(FLOOR(10*RAND()) AS INT64) daytrick, * EXCEPT(target_OE) FROM `statmike-mlops.digits.digits_source`)
SELECT * EXCEPT(daytrick), DATE_SUB(CURRENT_TIMESTAMP, INTERVAL daytrick DAY) AS update_time FROM A

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1610.51query/s]                        


""


Create Feature specification for each feature in the input source:

In [25]:
FEATURE_SPECS = []
for i in range(schema.shape[0]):
    FEATURE_SPECS.append(types.featurestore_service.ImportFeatureValuesRequest.FeatureSpec(id=schema['column_name'][i].lower()))

In [26]:
import_request = types.featurestore_service.ImportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    bigquery_source = types.BigQuerySource(input_uri='bq://statmike-mlops.digits.digits_featurestore_import'),
    feature_time_field = "update_time",
    feature_time = Timestamp().GetCurrentTime(),
    entity_id_field = "unique_id",
    feature_specs = FEATURE_SPECS,
    worker_count = 4,
)

In [27]:
importjob = clients['fs'].import_feature_values(import_request)

In [28]:
importjob.result()

imported_entity_count: 1797
imported_feature_value_count: 118602

---
## Serving Features

Retrieve a list of entity id's:

In [14]:
%%bigquery unique_id
SELECT unique_id FROM `statmike-mlops.digits.digits_featurestore_import`

Downloading: 100%|██████████| 1797/1797 [00:01<00:00, 1138.92rows/s]


In [15]:
id_values = list(unique_id['unique_id'])
id_values[0:5]

['a01bcbb6-f62c-413e-b850-3d1d879e88a1',
 'f3480418-b9eb-4131-9c8b-ab8250d36067',
 'e098d648-a0e6-49c0-b438-4d57b968c2ab',
 '11933c39-1140-4d2c-ba40-6a775fa2305c',
 '83c0d88a-aff1-470c-bc68-342ad3f7c7f6']

### Online - One (Current Value)

In [22]:
single = clients['fs_olserve'].read_feature_values(
    types.featurestore_online_service.ReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_id=id_values[0],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [23]:
print(list(item.id for item in single.header.feature_descriptors))

['p35', 'p5', 'p53', 'p29', 'p9', 'p20', 'p15', 'target_oe', 'p43', 'p48', 'p54', 'p41', 'p23', 'p33', 'p56', 'p0', 'p25', 'p16', 'p42', 'p58', 'p31', 'p44', 'p11', 'p2', 'p47', 'p57', 'p55', 'p49', 'p59', 'p40', 'p22', 'p32', 'p1', 'p34', 'p36', 'p14', 'p12', 'p19', 'p6', 'p10', 'p45', 'p52', 'p21', 'p62', 'p30', 'p51', 'p61', 'p8', 'p26', 'p13', 'p3', 'p18', 'p39', 'p17', 'p24', 'p28', 'p46', 'p38', 'target', 'p50', 'p4', 'p37', 'p7', 'p63', 'p60', 'p27']


In [24]:
print(list(item.value.double_value for item in single.entity_view.data))

[0.0, 6.0, 14.0, 12.0, 0.0, 4.0, 0.0, 0.0, 14.0, 0.0, 1.0, 1.0, 0.0, 6.0, 0.0, 0.0, 2.0, 0.0, 10.0, 0.0, 0.0, 15.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 3.0, 0.0, 0.0, 13.0, 1.0, 0.0, 16.0, 15.0, 0.0, 5.0, 16.0, 16.0, 16.0, 0.0, 8.0, 16.0, 5.0, 0.0, 14.0, 15.0, 9.0, 15.0, 0.0, 0.0, 0.0, 0.0, 3.0, 6.0, 0.0, 3.0, 15.0, 14.0, 0.0, 0.0, 13.0, 5.0]


#### Prediction

In [25]:
from google.cloud.aiplatform_v1beta1 import (PredictionServiceClient, EndpointServiceClient)
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

Function to retrieve online features for an entity and prepare them for prediction:

In [26]:
def fetch_single(entity_id):
    single = clients['fs_olserve'].read_feature_values(
        types.featurestore_online_service.ReadFeatureValuesRequest(
            entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
            entity_id=entity_id,
            feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
        )
    )
    
    newob = {}
    features = list(item.id for item in single.header.feature_descriptors)
    for e, f in enumerate(features):
        if f.startswith('p'):
            newob[f] = single.entity_view.data[e].value.double_value
    
    return newob

In [27]:
newob = fetch_single(id_values[0])

Pick an endpoint from those setup in this project (all have the same model parameters):

In [28]:
clients['endpoint'] = EndpointServiceClient(client_options=client_options)

In [29]:
list((ep.name, ep.display_name) for ep in clients['endpoint'].list_endpoints(parent=PARENT))

[('projects/691911073727/locations/us-central1/endpoints/4241669569355186176',
  'ENDPOINT_04_KERAS-DIGITS'),
 ('projects/691911073727/locations/us-central1/endpoints/8864051636897579008',
  'ENDPOINT_03_BQML-DIGITS'),
 ('projects/691911073727/locations/us-central1/endpoints/674818664477753344',
  '05C_AIP_DIGITS_20210902141249'),
 ('projects/691911073727/locations/us-central1/endpoints/2142992143000535040',
  '05C_AIP_DIGITS_20210902134247'),
 ('projects/691911073727/locations/us-central1/endpoints/732239559726727168',
  '05A_AIP_DIGITS_20210902123847'),
 ('projects/691911073727/locations/us-central1/endpoints/3811575804941303808',
  '05B_AIP_DIGITS_20210902112731'),
 ('projects/691911073727/locations/us-central1/endpoints/7922236364823724032',
  '05_AIP_DIGITS_20210901205004')]

In [30]:
ep = clients['endpoint'].list_endpoints(parent=PARENT).endpoints[2].name
ep

'projects/691911073727/locations/us-central1/endpoints/674818664477753344'

Python: Request a prediction for the entity's features:

In [31]:
clients['prediction'] = PredictionServiceClient(client_options=client_options)

In [32]:
response = clients['prediction'].predict(endpoint=ep, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [33]:
response

predictions {
  list_value {
    values {
      number_value: 0.974858642
    }
    values {
      number_value: 0.00159042131
    }
    values {
      number_value: 1.80285454e-07
    }
    values {
      number_value: 5.37076383e-10
    }
    values {
      number_value: 0.0224075858
    }
    values {
      number_value: 1.13798286e-08
    }
    values {
      number_value: 0.000751986227
    }
    values {
      number_value: 7.09668802e-06
    }
    values {
      number_value: 0.00038387996
    }
    values {
      number_value: 1.94259172e-07
    }
  }
}
deployed_model_id: "2532016949093728256"

REST: Request a prediction for the entity's features:

In [34]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [35]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{API_ENDPOINT}/v1/{ep}:predict

{
  "predictions": [
    [
      0.974858642,
      0.00159042131,
      1.80285454e-07,
      5.37076383e-10,
      0.0224075858,
      1.13798286e-08,
      0.000751986227,
      7.09668802e-06,
      0.00038387996,
      1.94259172e-07
    ]
  ],
  "deployedModelId": "2532016949093728256"
}


CLI: Request a prediction for the entity's features:

In [36]:
!gcloud beta ai endpoints predict {ep.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.974858642, 0.00159042131, 1.80285454e-07, 5.37076383e-10, 0.0224075858, 1.13798286e-08, 0.000751986227, 7.09668802e-06, 0.00038387996, 1.94259172e-07]]


### Online - Multi

In [37]:
multi = clients['fs_olserve'].streaming_read_feature_values(
    types.featurestore_online_service.StreamingReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_ids = id_values[0:3],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [38]:
for i in multi:
    print(i.entity_view.entity_id)
    print(list(item.value.double_value for item in i.entity_view.data))


[]
a01bcbb6-f62c-413e-b850-3d1d879e88a1
[14.0, 0.0, 0.0, 0.0, 13.0, 9.0, 13.0, 16.0, 3.0, 0.0, 0.0, 16.0, 16.0, 5.0, 14.0, 16.0, 0.0, 1.0, 16.0, 0.0, 15.0, 0.0, 5.0, 0.0, 6.0, 0.0, 0.0, 0.0, 4.0, 0.0, 2.0, 15.0, 14.0, 15.0, 0.0, 0.0, 5.0, 0.0, 12.0, 0.0, 0.0, 9.0, 0.0, 0.0, 1.0, 14.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 6.0, 3.0, 3.0, 6.0, 0.0, 0.0, 15.0, 1.0, 0.0, 10.0]
e098d648-a0e6-49c0-b438-4d57b968c2ab
[11.0, 0.0, 0.0, 0.0, 16.0, 7.0, 14.0, 16.0, 0.0, 0.0, 0.0, 16.0, 15.0, 5.0, 16.0, 16.0, 0.0, 0.0, 15.0, 0.0, 16.0, 0.0, 12.0, 0.0, 6.0, 0.0, 0.0, 4.0, 1.0, 0.0, 6.0, 4.0, 8.0, 8.0, 0.0, 0.0, 9.0, 0.0, 12.0, 0.0, 0.0, 9.0, 11.0, 0.0, 0.0, 15.0, 1.0, 15.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 5.0, 6.0, 5.0, 0.0, 0.0, 16.0, 1.0, 0.0, 15.0]
f3480418-b9eb-4131-9c8b-ab8250d36067
[5.0, 0.0, 0.0, 0.0, 14.0, 11.0, 13.0, 9.0, 1.0, 0.0, 0.0, 9.0, 11.0, 12.0, 15.0, 0.0, 0.0, 0.0, 2.0, 0.0, 12.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 2.0, 1.0, 0.0, 1.0, 0.0, 2.0, 10.0, 0.0, 0.

### Batch (For training or large scale prediction)

In [39]:
from google.cloud import bigquery

DESTINATION_DATASET = 'digits_training'

clients['bq'] = bigquery.Client()
dataset_id = "{}.{}".format(clients['bq'].project, DESTINATION_DATASET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = clients['bq'].create_dataset(dataset, exists_ok = True)

In [40]:
batch_request = types.featurestore_service.ExportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    snapshot_export = types.ExportFeatureValuesRequest.SnapshotExport(snapshot_time = Timestamp().GetCurrentTime()),
    destination = types.FeatureValueDestination(bigquery_destination = types.BigQueryDestination(output_uri='bq://statmike-mlops.digits_training.training')),
    feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*']))
)

In [41]:
batchjob = clients['fs'].export_feature_values(batch_request)

In [42]:
batchjob.result()

---
## Remove Resources
see notebook "XX - Cleanup"